In [1]:
%matplotlib inline
import importlib
import numpy as np
import pandas as pd
import json
import utils

In [2]:
importlib.reload(utils)

<module 'utils' from '/home/fei/Documents/projects/lyrics/stacked_lstm_statefull/utils.py'>

In [3]:
dat = pd.read_csv('../songdata.csv', usecols=['text']).drop_duplicates()

In [4]:
dat.head()

,text
0,"Look at her face, it's a wonderful face \nAnd..."
1,"Take it easy with me, please \nTouch me gentl..."
2,I'll never know why I had to go \nWhy I had t...
3,Making somebody happy is a question of give an...
4,Making somebody happy is a question of give an...


In [5]:
dat.loc[:, 'text'] += '(end)'

In [6]:
unique_chars = dat.loc[:, 'text'].apply(lambda x: list(set(x)))

In [7]:
unique_chars = unique_chars.sum()

In [6]:
unique_chars = pd.read_csv('unique_tokens', header=None, index_col=[0])

In [7]:
unique_chars.shape

(76, 1)

In [8]:
unique_chars.head()

,1
0,
0,\n
1,
2,!
3,""""
4,'


In [9]:
unique_chars = sorted(np.unique(unique_chars))

In [10]:
vocab_size = len(unique_chars)
vocab_size

76

In [11]:
unique_chars[:5]

['\n', ' ', '!', '"', "'"]

In [12]:
pd.Series(unique_chars).to_csv('unique_tokens', header=None)

In [17]:
utils.save_unique_tokens(unique_chars)

In [12]:
word2ind, ind2word = utils.get_index_word_map(unique_chars)

In [10]:
word2ind, ind2word = utils.load_index_word_map()

In [17]:
utils.save_index_word_map(word2ind, ind2word)

In [16]:
batches = dat.text.apply(len).value_counts()

In [18]:
batches.to_csv('data/batch_sizes.csv', header=None)

In [17]:
lengths = batches.index.tolist()

In [14]:
def scanning(array, input_length):
    batches = []
    text_length = array.shape[1]
    for i in range(text_length-input_length):
        batches.append(array[:, i:i+input_length+1])
    return np.concatenate(batches)

In [15]:
for ind, length in enumerate(lengths):
    if ind % 1000 == 0: print(ind)
    cleaned = dat.loc[dat.text.apply(len) == length, 'text']
    tokenised = utils.tokenise_cleaned_data(cleaned, word2ind)
    tokenised = np.array(tokenised.tolist())
    batched = scanning(tokenised, 300)
    pd.DataFrame(batched).to_csv('data/train_{}.csv'.format(length), index=False)

0
1000
2000
3000
